## Jupyterhub on Kubernetes

This notebook can be used to launch a standard instance of Jupyterhub which includes a proxy server, and an instance of Jupyterhub. 

This assumes you have a lauchfile in your working directory and have a Kubernetes cluster already loaded. 



### Print Configuration
Optionally you can print the configuration and common commands for your desired cluster. You can use this as a reference and copy and paste into the terminal.

In [1]:
!carme run list

carme: [INFO] These commands are currently installed:
create_service_account: "gcloud iam service-accounts create {g_service_account_name}\
  \ --display-name {g_service_account_name}"
create_key: "gcloud iam service-accounts keys create {cwd}/gcp/{g_authorization_file}\
  \ --iam-account {g_service_account_name}@{g_project}.iam.gserviceaccount.com"
get_policy: "gcloud iam service-accounts get-iam-policy {g_service_account_name}@{g_project}.iam.gserviceaccount.com\
  \ --format json > {cwd}gcp/policy.json"
set_policy: "gcloud iam service-accounts set-iam-policy {g_service_account_name}@{g_project}.iam.gserviceaccount.com\
  \ {cwd}/gcp/policy.json"
login: "gcloud auth login"
login_sa: "gcloud auth activate-service-account --key-file {cwd}/gcp/{g_authorization_file}"
install_helm: [install_helm0, install_helm1, install_helm2, install_helm3]
install_helm0: "curl https://raw.githubusercontent.com/kubernetes/helm/master/scripts/get\
  \ | bash"
install_helm1: "kubectl --namespace kube-syst

In [26]:
#Clone the Jupyterhub repo.
!carme run clone_repo

carme: [INFO] Running the command: clone_repo
carme: [INFO] Template: helm repo add jupyterhub {jup_helm_repo} && helm repo update
carme: [INFO] Values: helm repo add jupyterhub https://jupyterhub.github.io/helm-chart/ && helm repo update
Executing clone_repo:
 helm repo add jupyterhub https://jupyterhub.github.io/helm-chart/ && helm repo update
"jupyterhub" has been added to your repositories
Hang tight while we grab the latest from your chart repositories...
...Skip local chart repository
...Successfully got an update from the "jupyterhub" chart repository
...Successfully got an update from the "stable" chart repository
Update Complete. ⎈ Happy Helming!⎈ 


## Download Helm Repo

In [27]:
#This will install the jupyterhub instance.
!carme run install_jupyterhub


carme: [INFO] Running the command: install_jupyterhub
carme: [INFO] Template: helm install jupyterhub/jupyterhub --version={jup_version} --name={jup_releasename} --namespace={jup_namespace} -f {jup_config}
carme: [INFO] Values: helm install jupyterhub/jupyterhub --version=v0.6 --name=techfund2018 --namespace=techfund2018 -f /Users/jasonkuruzovich/githubdesktop/0_class/techfund_google/jupyter/jupyterhub/config.yaml
Executing install_jupyterhub:
 helm install jupyterhub/jupyterhub --version=v0.6 --name=techfund2018 --namespace=techfund2018 -f /Users/jasonkuruzovich/githubdesktop/0_class/techfund_google/jupyter/jupyterhub/config.yaml
Error: timed out waiting for the condition


In [30]:
#This will check on the status of Jupyterhub.
!carme run describe_jupyterhub 

carme: [INFO] Running the command: describe_jupyterhub
carme: [INFO] Template: kubectl --namespace={jup_namespace} get pod
carme: [INFO] Values: kubectl --namespace=techfund2018v2 get pod
Executing describe_jupyterhub:
 kubectl --namespace=techfund2018v2 get pod
NAME                                         READY     STATUS    RESTARTS   AGE
hub-55bbd4ffb8-l6vck                         1/1       Running   0          3m
pre-pull-techfund2018v2-1-1524111096-z44nq   1/1       Running   0          8m
pre-pull-techfund2018v2-1-1524111400-r7cgj   1/1       Running   0          3m
proxy-75596487b4-44bcp                       2/2       Running   0          3m


In [31]:
#Get the ip/url. 
#cf=ku.get_jupyterhub_ip(cf)
!carme run get_ip

carme: [INFO] Running the command: get_ip
carme: [INFO] Template: kubectl --namespace={jup_namespace} get svc proxy-public
carme: [INFO] Values: kubectl --namespace=techfund2018v2 get svc proxy-public
Executing get_ip:
 kubectl --namespace=techfund2018v2 get svc proxy-public
NAME           TYPE           CLUSTER-IP     EXTERNAL-IP     PORT(S)                      AGE
proxy-public   LoadBalancer   10.15.253.15   35.190.129.37   80:31988/TCP,443:31091/TCP   3m


## Update Authorization
When just starting out, the default configuration launches with no password.  While there is still a username and login screen.
```
!carme app jupyter jup_dummy_auth
!carme app jupyter jup_google_auth
!carme app jupyter jup_github_auth
```

In [ ]:
!carme app jupyter jup_dummy_auth


In [ ]:
!carme app jupyter jup_admin

### Update the Singleuser Container
This can allow you to utilize a different container. 


In [ ]:
!carme app jupyter jup_singleuser

In [ ]:
!carme app jupyter show_config 


In [41]:
#Upgrading Jupyterhub 
!carme run upgrade_jupyterhub 

carme: [INFO] Running the command: upgrade_jupyterhub
carme: [INFO] Template: helm upgrade {jup_namespace} jupyterhub/jupyterhub --version={jup_version} -f ../apps/jupyterhub/config.yaml
carme: [INFO] Values: helm upgrade techfund2018v2 jupyterhub/jupyterhub --version=v0.6 -f ../apps/jupyterhub/config.yaml
Executing upgrade_jupyterhub:
 helm upgrade techfund2018v2 jupyterhub/jupyterhub --version=v0.6 -f ../apps/jupyterhub/config.yaml
Release "techfund2018v2" has been upgraded. Happy Helming!
LAST DEPLOYED: Sat Apr 21 21:40:50 2018
NAMESPACE: techfund2018v2
STATUS: DEPLOYED

RESOURCES:
==> v1beta1/Deployment
NAME   DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
hub    1        1        1           0          2d
proxy  1        1        1           1          2d

==> v1/Secret
NAME        TYPE    DATA  AGE
hub-secret  Opaque  1     2d

==> v1/ConfigMap
NAME                DATA  AGE
hub-config          31    2d
nginx-proxy-config  1     2d

==> v1/PersistentVolumeClaim
NAME        STATUS  V

In [39]:
!kubectl delete pod hub-699cb97d7f-hn46d

Error from server (NotFound): pods "hub-699cb97d7f-hn46d" not found


In [38]:
!kubectl get pods --all-namespaces

NAMESPACE        NAME                                                        READY     STATUS    RESTARTS   AGE
kube-system      event-exporter-v0.1.7-5c4d9556cf-tcnls                      2/2       Running   0          2d
kube-system      fluentd-gcp-v2.0.9-r2gfm                                    2/2       Running   0          2d
kube-system      heapster-v1.4.3-7994889f85-zhlsc                            3/3       Running   0          2d
kube-system      kube-dns-778977457c-lprkv                                   3/3       Running   0          2d
kube-system      kube-dns-autoscaler-7db47cb9b7-94jcp                        1/1       Running   0          2d
kube-system      kube-proxy-gke-techfunsp2018v2-default-pool-bd679e73-hghz   1/1       Running   0          2d
kube-system      kubernetes-dashboard-768854d6dc-dg9ms                       1/1       Running   3          2d
kube-system      l7-default-backend-6497bcdb4d-mkr4p                         1/1       Running   0          2d


## Cleanup the Installation
This will cleanup the installation, deleting the instance of Jupyterhub.

In [28]:
#Delete Jupyterhub with Helm.
!carme run delete_jupyterhub

carme: [INFO] Running the command: delete_jupyterhub
carme: [INFO] Template: helm delete {jup_releasename} --purge
carme: [INFO] Values: helm delete techfund2018 --purge
Executing delete_jupyterhub:
 helm delete techfund2018 --purge
Error: release: "techfund2018" not found


In [29]:
#Delete the namespace
!carme run delete_namespace

carme: [INFO] Running the command: delete_namespace
carme: [INFO] Template: kubectl delete namespace {jup_namespace}
carme: [INFO] Values: kubectl delete namespace techfund2018
Executing delete_namespace:
 kubectl delete namespace techfund2018
namespace "techfund2018" deleted
